In [1]:
# @hidden_cell
# The project token is an authorization token that is used to access project resources like data sources, connections, and used by platform APIs.
from project_lib import Project
project = Project(spark.sparkContext, '0062b6b9-363a-48ea-8273-51b639cd0891', 'p-577ee47309c0be1153ed7c39d05c265f9295d159')
pc = project.project_context


Waiting for a Spark session to start...
Spark Initialization Done! ApplicationId = app-20200402091815-0000
KERNEL_ID = 93816d4d-bfbc-4bfd-8260-2608561fc5ef


In [40]:
#import training data from IBM cloud

import ibmos2spark
# @hidden_cell
credentials = {

}

configuration_name = 'os_c63666ea1c9d403e811d4c86742ad623_configs'
cos = ibmos2spark.CloudObjectStorage(sc, credentials, configuration_name, 'bluemix_cos')

from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()
df = spark.read\
  .format('org.apache.spark.sql.execution.datasources.csv.CSVFileFormat')\
  .option('header', 'true')\
  .load(cos.url('train.csv', 'bostonhousing-donotdelete-pr-f7plktwicr9r0f'), inferSchema='true')


In [41]:
#Import the test set to be submitted to Kaggle as Spark dataframe. Use schema from the train set.
df_test = spark.read\
  .format('org.apache.spark.sql.execution.datasources.csv.CSVFileFormat')\
  .option('header', 'true')\
  .load(cos.url('test.csv', 'bostonhousing-donotdelete-pr-f7plktwicr9r0f'), inferSchema= 'true')#schema=df_data_train.schema)
#df_test.take(1)


#Import the sample submission for Kaggle as pandas dataframe
import types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.


body = client_c63666ea1c9d403e811d4c86742ad623.get_object(Bucket='bostonhousing-donotdelete-pr-f7plktwicr9r0f',Key='sample_submission.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

sample_sub = pd.read_csv(body)
sample_sub.head(1)


,Id,SalePrice
0,1461,169277.052498


Check if there are missing values (there are none)

In [42]:
import pyspark.sql.functions as F

def count_missings(spark_df,sort=True):
    df = spark_df.select([F.count(F.when(F.isnan(c) | F.isnull(c), c)).alias(c) for (c,c_type) in spark_df.dtypes if c_type not in ('timestamp', 'string', 'date')]).toPandas()

    if len(df) == 0:
        print("There are no any missing values!")
        return None

    if sort:
        return df.rename(index={0: 'count'}).T.sort_values("count",ascending=False)

    return df

count_missings(df_test, False)
count_missings(df, False)

,Id,MSSubClass,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,...,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SalePrice
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [43]:
# Create two separate lists of categorical and numerical features
columns_cat = [item[0] for item in df.dtypes if item[1].startswith('string')]
columns_num = [item[0] for item in df.dtypes if item[1].startswith('int') | item[1].startswith('double')]

print(columns_cat)
print(columns_num)

['MSZoning', 'LotFrontage', 'Street', 'Alley', 'LotShape', 'LandContour', 'Utilities', 'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType', 'HouseStyle', 'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType', 'MasVnrArea', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'Heating', 'HeatingQC', 'CentralAir', 'Electrical', 'KitchenQual', 'Functional', 'FireplaceQu', 'GarageType', 'GarageYrBlt', 'GarageFinish', 'GarageQual', 'GarageCond', 'PavedDrive', 'PoolQC', 'Fence', 'MiscFeature', 'SaleType', 'SaleCondition']
['Id', 'MSSubClass', 'LotArea', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd', 'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'TotRmsAbvGrd', 'Fireplaces', 'GarageCars', 'GarageArea', 'WoodDeckSF', 'OpenPorchSF', '

In [44]:
# Replace "NA" in test set with 0
df_test = df_test.replace('NA', '0')
for col in columns_num:
    if (col == 'SalePrice') | (col == 'Id'):
        continue
    df_test = df_test.withColumn(col, df_test[col].cast('double'))
    df = df.withColumn(col, df[col].cast('double'))



Verify that the schema of the dataframe looks ok (features that are categorical should be strings, numeric features should be integers or double)

In [45]:
df_test.printSchema()

root
 |-- Id: integer (nullable = true)
 |-- MSSubClass: double (nullable = true)
 |-- MSZoning: string (nullable = true)
 |-- LotFrontage: string (nullable = true)
 |-- LotArea: double (nullable = true)
 |-- Street: string (nullable = true)
 |-- Alley: string (nullable = true)
 |-- LotShape: string (nullable = true)
 |-- LandContour: string (nullable = true)
 |-- Utilities: string (nullable = true)
 |-- LotConfig: string (nullable = true)
 |-- LandSlope: string (nullable = true)
 |-- Neighborhood: string (nullable = true)
 |-- Condition1: string (nullable = true)
 |-- Condition2: string (nullable = true)
 |-- BldgType: string (nullable = true)
 |-- HouseStyle: string (nullable = true)
 |-- OverallQual: double (nullable = true)
 |-- OverallCond: double (nullable = true)
 |-- YearBuilt: double (nullable = true)
 |-- YearRemodAdd: double (nullable = true)
 |-- RoofStyle: string (nullable = true)
 |-- RoofMatl: string (nullable = true)
 |-- Exterior1st: string (nullable = true)
 |-- Exter

In [46]:
#Replace the PoolArea feature with a Has Pool feature, there are very few houses with pools in the dataset (only 7 of 1460)
#from pyspark.ml.feature import Binarizer

#df = df.withColumn("PoolArea", df.PoolArea.cast("double"))

#binarizer = Binarizer(threshold=0.0, inputCol='PoolArea', outputCol='Has_Pool')

#df = binarizer.transform(df)

#df = df.drop('PoolArea')

In [47]:
# Find correlation between numerical features and the sale price

from operator import itemgetter

corr_list = []

for column in columns_num:
    corr_val = df.corr('SalePrice', column)
    corr_list.append((column, corr_val))

corr_list = sorted(corr_list, key=itemgetter(1))


num_cols = []

# Create list of numerical cols with a correlation above 0.35 with the sale price
for col, val in corr_list:
    if col == 'SalePrice':
        continue
    if val > 0.35:
        num_cols.append(col)

print(num_cols)
    
    

['BsmtFinSF1', 'Fireplaces', 'YearRemodAdd', 'YearBuilt', 'TotRmsAbvGrd', 'FullBath', '1stFlrSF', 'TotalBsmtSF', 'GarageArea', 'GarageCars', 'GrLivArea', 'OverallQual']


In [48]:
from pyspark.ml import Pipeline
from pyspark.ml.regression import GBTRegressor
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.feature import StringIndexer
from pyspark.ml.feature import OneHotEncoderEstimator
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder

cat_cols = ['MSZoning', 'Neighborhood' , 'HouseStyle', 'CentralAir']

cols_for_model = num_cols + cat_cols + ['SalePrice']

df_red = df[cols_for_model]

stages = []

# Convert categorical strings to index values
for cat_col in cat_cols:
    indexer = StringIndexer(inputCol=cat_col, outputCol= cat_col +'_idx')
    onehot = OneHotEncoderEstimator(inputCols=[indexer.getOutputCol()], outputCols= [cat_col + '_dummy'])
    stages += [indexer, onehot]



# Assemble predictors into a single column 
assembler = VectorAssembler(inputCols=num_cols, outputCol='features')

stages += [assembler]

h_train, h_test = df_red.randomSplit([0.8, 0.2], seed=1)

# Print out columns to be used in prediction
print(h_train.columns)


['BsmtFinSF1', 'Fireplaces', 'YearRemodAdd', 'YearBuilt', 'TotRmsAbvGrd', 'FullBath', '1stFlrSF', 'TotalBsmtSF', 'GarageArea', 'GarageCars', 'GrLivArea', 'OverallQual', 'MSZoning', 'Neighborhood', 'HouseStyle', 'CentralAir', 'SalePrice']


In [49]:
# Create a Gradient boosted tree regressor
gbt = GBTRegressor(featuresCol='features',
                           labelCol='SalePrice',
                           seed=1,
                           stepSize = 0.05,
                           maxIter = 100
                           )


stages += [gbt]


# Create a pipeline
pipeline = Pipeline(stages=stages)

# Create the evaluator, evaluate on Root mean square error
pip_evaluator = RegressionEvaluator(
    labelCol='SalePrice', predictionCol='prediction', metricName='rmse')

# Create a parameter grid to try different parameters
paramGrid = (ParamGridBuilder()
             .addGrid(gbt.maxDepth, [4, 5])
             .build())


# Use cross validation to find the best model
cv = CrossValidator(estimator=pipeline, estimatorParamMaps=paramGrid, evaluator=pip_evaluator, numFolds=5, seed=1)

cvModel = cv.fit(h_train)


In [50]:
# Get the best model from cross validation
best_model = cvModel.bestModel

# Look at the stages in the best model
print(best_model.stages)

# select stage 9 (the GBTRegressor) of the best model and see info regarding which depth the model has chosen, this can be used on any parameter
print(best_model.stages[9].explainParam('maxDepth'))

# Generate predictions on testing data using the best model then calculate RMSE
predictions = best_model.transform(h_test)
rmse = pip_evaluator.evaluate(predictions)

print("Root Mean Squared Error (RMSE) on test data = %g" % rmse)



[StringIndexer_adcc7d7e8e8a, OneHotEncoderEstimator_9797e49b61e3, StringIndexer_0784cb5836fb, OneHotEncoderEstimator_66d04ff38864, StringIndexer_2b9b37d1a9a6, OneHotEncoderEstimator_08855e3f82dc, StringIndexer_900c06aaea2c, OneHotEncoderEstimator_2c9724a2be1d, VectorAssembler_4aca3cec6fef, GBTRegressionModel (uid=GBTRegressor_b10fb6198a6b) with 100 trees]
maxDepth: Maximum depth of the tree. (>= 0) E.g., depth 0 means 1 leaf node; depth 1 means 1 internal node + 2 leaf nodes. (default: 5, current: 4)
Root Mean Squared Error (RMSE) on test data = 29068.5


Predict the prices for the test set and create a submission file for Kaggle

In [51]:
# Choose the columns
df_test_red = df_test[num_cols + cat_cols]
df_test = df_test.fillna(0, subset=columns_num)

# Make prediction
kaggle_pred = best_model.transform(df_test)

kaggle_pred = kaggle_pred.withColumn('Id', kaggle_pred['Id'].cast('int'))

# Select only the required columns and convert to pandas dataframe for export
kp_pandas = kaggle_pred.select('prediction', 'Id').toPandas()

kp_pandas['SalePrice'] = kp_pandas.prediction
kp_pandas = kp_pandas.drop(columns=['prediction'])
print(kp_pandas.head())

     Id      SalePrice
0  1461  118917.510767
1  1462  148098.012267
2  1463  194026.610188
3  1464  192868.505328
4  1465  193203.012501


Save the data to a csv-file to be able to submit the results to Kaggle

In [52]:
project.save_data("submission8.csv", kp_pandas.to_csv(index=False))


{'file_name': 'submission8.csv',
 'message': 'File saved to project storage.',
 'bucket_name': 'bostonhousing-donotdelete-pr-f7plktwicr9r0f',
 'asset_id': 'd8ff00d2-bb17-4f24-b46b-c0adaa1ea019'}